<a href="https://colab.research.google.com/github/rjnakano/manejo-inventarios-pdg/blob/main/Notebooks/clusterboot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
pip install --upgrade kmodes

Requirement already up-to-date: kmodes in /usr/local/lib/python3.7/dist-packages (0.11.0)


In [21]:
from google.colab import drive
import pathlib
import pandas as pd
import numpy as np
from pylab import *

from sklearn.preprocessing import StandardScaler

from kmodes.kprototypes import KPrototypes

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import seaborn as sns
%matplotlib inline
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()

from plotnine import *
import plotnine

In [22]:
plot_kwds = {'alpha' : 0.5, 's' : 1, 'linewidths':0}

In [23]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [24]:
# Dataset de inventarios sin dimensiones de los productos
data1 = pd.read_csv("/content/gdrive/MyDrive/TDGdata/df042421.csv")

In [25]:
data1['AISLE']=data1['AISLE'].astype(str)

In [26]:
data1.dtypes

AISLE        object
IG           object
HTS          object
PLANNING     object
ABC          object
UOM          object
CAT          object
COSTX       float64
QTYVAR      float64
dtype: object

In [8]:
#data1['ID']=data1.index

In [27]:
data1.head(5)

,AISLE,IG,HTS,PLANNING,ABC,UOM,CAT,COSTX,QTYVAR
0,17,IG56,HTS119,1/1,B,PC,CAT17,-0.029671,-0.037167
1,17,IG56,HTS119,1/1,B,PC,CAT17,0.092655,-0.006905
2,17,IG56,HTS119,1/1,C,PC,CAT17,0.266138,0.020477
3,17,IG56,HTS119,1/1,B,PC,CAT17,-0.024812,-0.044242
4,15,IG56,HTS119,1/1,C,PC,CAT17,-0.077628,-0.046455


In [28]:
# scikit-learn bootstrap
from sklearn.utils import resample

In [29]:
data1.shape[0]

16244

In [30]:
# prepare bootstrap sample
boot = resample(data1, replace=True, n_samples=data1.shape[0], random_state=1)

In [31]:
boot.head(5)

,AISLE,IG,HTS,PLANNING,ABC,UOM,CAT,COSTX,QTYVAR
13349,13,IG46,HTS119,1/1,C,PR,CAT17,0.217599,0.002996
235,17,IG45,HTS24,1/1,A,BG,CAT17,0.005390,-0.039992
12172,32,IG61,HTS6,1/1,A,BG,CAT14,0.354152,-0.033191
5192,2,IG38,HTS61,1/1,A,SQFT,CAT8,-0.052529,-0.143812
16127,22,IG17,HTS95,1/0,C,BX,CAT12,0.349906,-0.033191


In [32]:
boot.shape

(16244, 9)

In [33]:
boot.duplicated()

13349    False
235      False
12172    False
5192     False
16127    False
         ...  
4228     False
10630     True
3549      True
5676     False
13250     True
Length: 16244, dtype: bool

In [35]:
boot[boot.index==1478]

,AISLE,IG,HTS,PLANNING,ABC,UOM,CAT,COSTX,QTYVAR
1478,21,IG47,HTS119,1/1,B,PC,CAT17,0.193305,0.003555
1478,21,IG47,HTS119,1/1,B,PC,CAT17,0.193305,0.003555
1478,21,IG47,HTS119,1/1,B,PC,CAT17,0.193305,0.003555
1478,21,IG47,HTS119,1/1,B,PC,CAT17,0.193305,0.003555
1478,21,IG47,HTS119,1/1,B,PC,CAT17,0.193305,0.003555


In [36]:
catColumnsPos = [data1.columns.get_loc(col) for col in list(data1.select_dtypes('object').columns)]

In [37]:
catColumnsPos

[0, 1, 2, 3, 4, 5, 6]

In [38]:
# Fit the cluster 
k = 4
gamma = 0.5
import time
start_time = time.time()
kprototype = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = gamma, random_state = 0)
kprototype.fit_predict(data1, categorical = catColumnsPos)
print("--- %s seconds ---" % (time.time() - start_time))

--- 71.64014315605164 seconds ---


In [39]:
print(kprototype.cluster_centroids_)

[['0.002726388751717149' '-0.03595785577407407' '2' 'IG45' 'HTS119' '1/1'
  'A' 'BX' 'CAT17']
 ['-3.6487292748298596' '-0.11901373330295531' '37' 'IG64' 'HTS15' '1/1'
  'A' 'BX' 'CAT14']
 ['1.784357513065899' '36.7170829887051' '2' 'IG38' 'HTS61' '1/1' 'A'
  'SQFT' 'CAT8']
 ['2.79799290946075' '0.5053389823974501' '37' 'IG64' 'HTS15' '1/1' 'A'
  'BX' 'CAT14']]


In [40]:
data1['Cluster'] = kprototype.labels_

In [41]:
boot.dtypes

AISLE        object
IG           object
HTS          object
PLANNING     object
ABC          object
UOM          object
CAT          object
COSTX       float64
QTYVAR      float64
dtype: object

In [42]:
# Fit the cluster 
k = 4
gamma = 0.5
import time
start_time = time.time()
kprototypebs = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = gamma, random_state = 0)
kprototypebs.fit_predict(boot, categorical = catColumnsPos)
print("--- %s seconds ---" % (time.time() - start_time))

--- 76.18300867080688 seconds ---


In [ ]:
print(kprototypebs.cluster_centroids_)

[['0.05565383927700991' '-0.022394882604889192' '12' 'IG46' 'HTS119'
  '1/1' 'A' 'PC' 'CAT17']
 ['-0.06438038500544505' '-0.053530134537021434' '2' 'IG14' 'HTS15' '1/1'
  'C' 'BX' 'CAT8']
 ['-4.199122494971278' '-0.11922423118019004' '37' 'IG61' 'HTS15' '1/1'
  'A' 'BX' 'CAT14']
 ['2.9289436099474213' '1.0022811424258908' '37' 'IG64' 'HTS15' '1/1' 'A'
  'BX' 'CAT14']]


In [43]:
boot['Cluster'] = kprototypebs.labels_

In [55]:
data1['ID']=data1.index

In [138]:
orig0 = {999999}

In [139]:
orig0.update(data1[data1['Cluster']==0].index)

In [140]:
orig0.discard(999999)

In [92]:
orig1 = {999999}

In [93]:
orig1.update(data1[data1['Cluster']==1].index)

In [94]:
orig1.discard(999999)

In [100]:
orig2 = {999999}

In [101]:
orig2.update(data1[data1['Cluster']==2].index)

In [102]:
orig2.discard(999999)

In [104]:
orig3 = {999999}

In [105]:
orig3.update(data1[data1['Cluster']==3].index)

In [106]:
orig3.discard(999999)

In [112]:
boot0 = {999999}

In [113]:
boot0.update(boot[boot['Cluster']==0].index)

In [114]:
boot0.discard(999999)

In [116]:
boot1 = {999999}

In [117]:
boot1.update(boot[boot['Cluster']==1].index)

In [118]:
boot1.discard(999999)

In [120]:
boot2 = {999999}

In [121]:
boot2.update(boot[boot['Cluster']==2].index)

In [122]:
boot2.discard(999999)

In [124]:
boot3 = {999999}

In [125]:
boot3.update(boot[boot['Cluster']==3].index)

In [126]:
boot3.discard(999999)

In [146]:
# orig0
max(len(orig0 & boot0) / len(orig0 | boot0), len(orig0 & boot1) / len(orig0 | boot1), len(orig0 & boot2) / len(orig0 | boot2), len(orig0 & boot3) / len(orig0 | boot3))

0.33989985505336673

In [147]:
# orig1
max(len(orig1 & boot0) / len(orig1 | boot0), len(orig1 & boot1) / len(orig1 | boot1), len(orig1 & boot2) / len(orig1 | boot2), len(orig1 & boot3) / len(orig1 | boot3))

0.5590062111801242

In [148]:
# orig2
max(len(orig2 & boot0) / len(orig2 | boot0), len(orig2 & boot1) / len(orig2 | boot1), len(orig2 & boot2) / len(orig2 | boot2), len(orig2 & boot3) / len(orig2 | boot3))

0.008547008547008548

In [149]:
# orig3
max(len(orig3 & boot0) / len(orig3 | boot0), len(orig3 & boot1) / len(orig3 | boot1), len(orig3 & boot2) / len(orig3 | boot2), len(orig3 & boot3) / len(orig3 | boot3))

0.5419354838709678

In [ ]:
# from sklearn.metrics import jaccard_score

In [ ]:
# jaccard_score(data1['Cluster'], boot['Cluster'], average=None)

array([0.4904868 , 0.02772796, 0.        , 0.02278703])